In [72]:
import numpy as np
import matplotlib.pyplot as plt


In [106]:
J = 1.0


def find_neigbours(size):
    a_neighbours = np.zeros((size,2),dtype=int)
    for i in range(size):
        a_neighbours[i,0] = (i+1)%size
        a_neighbours[i,1] = (i-1)%size

    def random_nonself_non_neighbour(N,i):
        nbs = a_neighbours[i]
        while True:
            r = np.random.randint(0,size)
            if r != i and r not in nbs:
                return r



    b_neighbours = np.zeros((size,4),dtype=int)
    for i in range(size):
        b_neighbours[i,0] = (i+1)%size
        b_neighbours[i,1] = (i-1)%size
        b_neighbours[i,2] = random_nonself_non_neighbour(size,i)
        b_neighbours[i,3] = random_nonself_non_neighbour(size,i)

    return a_neighbours, b_neighbours

def order_param(lattice):
    return np.sum(lattice)/len(lattice)

def E_full(lattice):
    return -J * np.sum(lattice * np.roll(lattice, 1, axis=0))

def E(site, lrest):
    return -J * site * lrest.sum()

def x_fun(site, lrest, T):
    return np.exp(2*E(site,lrest)*(1/T))

def flip(lattice, x):
    lattice[x] *= -1

def flip_maybe(lattice, neighbours, T):
    x = np.random.randint(0, size)
    
    indexs = neighbours[x]

    li = lattice[indexs]

    xf = x_fun(lattice[x], li, T)

    if np.random.random() < xf: # maybe .copy() or .ravel()
        flip(lattice, x)

def Jspace(n):
    return np.concatenate((np.linspace(0.1,1,n-1, endpoint=False),np.linspace(1,10,n, endpoint=True)))

def run(neighbours, T, n_runs):
    lattice = np.random.choice([-1,1],size)
    for i in range(n_runs):
        flip_maybe(lattice, neighbours, T)
    return lattice



def sweep_T(size, n_runs):
    a_neighbours, b_neighbours = find_neigbours(size)
    for nei in [a_neighbours, b_neighbours]:
        for T in Jspace(10):
            lattice = run(nei, T, n_runs)
            print(f"T: {T:.1f} <s>: {order_param(lattice):.3f}")
            # plt.imshow(np.repeat(lattice, 50,axis=0).reshape(size,50).T,cmap='gray',interpolation='none')
            # plt.show()




def binarysearch_4_T(size, n_runs):
    a_neighbours, b_neighbours = find_neigbours(size)
    max_t = 5
    min_t = 0.1
    T = (max_t+min_t)/2
    for i in range(20):
        T = (max_t+min_t)/2
        lattice = run(b_neighbours, T, n_runs)

        op = np.abs(order_param(lattice))
        if op < 0.9:
            max_t = T
        else:
            min_t = T


    print(f"Found T: {T:.3f} <s>: {op:.3f}")
    less, more = T-0.1, T+0.1
    more_lattice = run(b_neighbours, more, n_runs)
    less_lattice = run(b_neighbours, less, n_runs)
    more_op = np.abs(order_param(more_lattice))
    less_op = np.abs(order_param(less_lattice))
    print("T-0.1 <s>: ", less_op)
    print("T+0.1 <s>: ", more_op)
    print("")

sizes = [100, 500, 1000, 5000, 10000, 50000]
for size in sizes:
    n_runs = 10000
    print(f"size: {size}")
    binarysearch_4_T(size, n_runs)

size: 100
Found T: 2.059 <s>: 0.980
T-0.1 <s>:  0.96
T+0.1 <s>:  0.78

size: 500
Found T: 1.842 <s>: 0.880
T-0.1 <s>:  0.972
T+0.1 <s>:  0.748

size: 1000
Found T: 1.443 <s>: 0.978
T-0.1 <s>:  0.852
T+0.1 <s>:  0.602

size: 5000
Found T: 0.100 <s>: 0.037


C:\Users\jakob\AppData\Local\Temp\ipykernel_14672\3572633068.py:38: RuntimeWarning: overflow encountered in exp
  return np.exp(2*E(site,lrest)*(1/T))


T-0.1 <s>:  0.0012
T+0.1 <s>:  0.036

size: 10000
Found T: 0.100 <s>: 0.008
T-0.1 <s>:  0.0028
T+0.1 <s>:  0.0558

size: 50000
Found T: 0.100 <s>: 0.000
T-0.1 <s>:  0.00944
T+0.1 <s>:  0.00436



In [103]:
sweep_T(100, 100000)

T: 0.1 <s>: -1.000
T: 0.2 <s>: 1.000
T: 0.3 <s>: 1.000
T: 0.4 <s>: 0.440
T: 0.5 <s>: -0.480
T: 0.6 <s>: 0.000
T: 0.7 <s>: 0.020
T: 0.8 <s>: 0.020
T: 0.9 <s>: 0.080
T: 1.0 <s>: 0.220
T: 2.0 <s>: -0.220
T: 3.0 <s>: 0.200
T: 4.0 <s>: -0.140
T: 5.0 <s>: 0.300
T: 6.0 <s>: -0.080
T: 7.0 <s>: -0.040
T: 8.0 <s>: 0.020
T: 9.0 <s>: 0.120
T: 10.0 <s>: -0.060
T: 0.1 <s>: -1.000
T: 0.2 <s>: 1.000
T: 0.3 <s>: -1.000
T: 0.4 <s>: 1.000
T: 0.5 <s>: 1.000
T: 0.6 <s>: 1.000
T: 0.7 <s>: -1.000
T: 0.8 <s>: -1.000
T: 0.9 <s>: 1.000
T: 1.0 <s>: 1.000
T: 2.0 <s>: -0.900
T: 3.0 <s>: 0.000
T: 4.0 <s>: -0.220
T: 5.0 <s>: 0.080
T: 6.0 <s>: -0.020
T: 7.0 <s>: 0.200
T: 8.0 <s>: -0.080
T: 9.0 <s>: -0.200
T: 10.0 <s>: 0.040
